In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

# 🚀 JEPA: Predicting in Abstract Space — Why Pixels Don't Matter
**Notebook 4 of 6 — World Action Models Series | Vizuara**

*Estimated time: ~40 minutes*

In this notebook, we will build **I-JEPA (Image-based Joint Embedding Predictive Architecture)** from scratch and discover why predicting in abstract representation space is far more powerful than predicting pixels.

In [ ]:
# --- Setup and GPU Check ---
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if device.type == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# Reproducibility
torch.manual_seed(42)
np.random.seed(42)

%matplotlib inline

In [ ]:
!pip install scikit-learn -q

## 1. Why Does This Matter?

Let us start with a simple thought experiment.

When you imagine catching a ball thrown towards you, what goes through your mind? Do you simulate every blade of grass on the field, every cloud in the sky, the exact pattern of light reflecting off the ball's surface?

Of course not. You predict **trajectory, speed, and landing position** — the things that actually matter. You filter out all the irrelevant visual details and reason about the **abstract essence** of what is happening.

This is precisely the insight behind **JEPA — Joint Embedding Predictive Architecture**, proposed by Yann LeCun. Instead of predicting raw pixels (like a VAE or diffusion model would), JEPA predicts in **abstract representation space**. It learns to capture what matters and ignore what does not.

> **By the end of this notebook, you will build I-JEPA from scratch and watch it learn powerful visual representations without ever generating a single pixel.**

We will:
1. Understand why pixel-level prediction wastes capacity
2. Walk through the JEPA mathematics step by step
3. Implement patch embeddings, a simplified Vision Transformer encoder, a predictor, and the EMA target encoder
4. Train I-JEPA on CIFAR-10 and visualize the learned representations with t-SNE

Let us begin.

## 2. Building Intuition — Why Predict in Abstract Space?

### The Problem with Pixel Prediction

Generative models like VAEs and diffusion models learn by reconstructing pixels. They must predict every single pixel value — the exact shade of grass, the precise pattern of shadows, the specific texture of a wall.

Think about what that means: a model that is trying to understand "there is a dog in this image" must also spend capacity learning "the third blade of grass is 2 pixels wide and has RGB value (34, 120, 45)."

This is enormously wasteful. The model wastes representational power on **surface-level details** that carry little semantic meaning.

### JEPA's Core Insight

JEPA says: **predict in abstract space, not pixel space.**

Instead of forcing the model to reconstruct the raw image, we:
1. Encode the image into a compact representation (an embedding)
2. Predict **embeddings** of missing parts from the embeddings of visible parts

The key difference: embeddings naturally discard irrelevant details and preserve semantic meaning.

### An Analogy

Imagine you are a student and your professor asks you to summarize a chapter from a textbook.

- **Pixel prediction** is like being asked to copy the chapter word-for-word — you would reproduce everything, including typos, formatting quirks, and irrelevant footnotes.
- **JEPA (abstract prediction)** is like being asked to write a summary — you capture the **meaning** and ignore the surface noise.

Which approach demonstrates deeper understanding? The summary, of course. And that is what JEPA optimizes for.

### 🤔 Think About This

Before we dive into the mathematics, consider this question:

> *If a robot needs to pick up a mug from a table, does it need to predict the exact reflection pattern on the mug's surface? What information actually matters for the task — and what is just noise?*

The information that matters: the mug's **position**, **orientation**, **shape**, and **size**. The reflection pattern? Completely irrelevant. Yet a pixel-level generative model would spend significant capacity modeling those reflections.

JEPA learns to focus on what matters by predicting in a space where irrelevant details have already been discarded.

### How I-JEPA Works at a High Level

Here is the recipe for I-JEPA:

1. Take an image and split it into **patches** (like a grid of small squares)
2. **Mask out** some patches (the "target" patches)
3. **Encode** the visible patches using a context encoder
4. **Encode** the masked patches using a separate target encoder (which the model does not train directly via gradients)
5. Use a **predictor** to predict the target embeddings from the context embeddings
6. The loss is simply: how close are the predicted embeddings to the actual target embeddings?

The target encoder is updated slowly using an **Exponential Moving Average (EMA)** of the context encoder's weights. This is the critical trick that prevents the model from finding a trivial shortcut (like mapping everything to zero).

Now let us formalize this.

## 3. The Mathematics of JEPA

Let us walk through the mathematics step by step. We will define each component and then immediately explain what it means computationally.

### 3.1 Context Encoder

$$s_x = E_\theta(x)$$

Here, $x$ represents the **visible patches** of the image, $E_\theta$ is our context encoder (a Vision Transformer with learnable parameters $\theta$), and $s_x$ is the resulting embedding — a vector that captures the semantic content of the visible patches.

**Computationally:** we take the visible patches, pass each one through a shared encoder network, and get out embedding vectors. If we have 10 visible patches and our embedding dimension is 128, then $s_x$ is a matrix of shape $(10, 128)$.

### 3.2 Target Encoder

$$s_y = E_{\bar{\theta}}(y)$$

Here, $y$ represents the **masked (target) patches**, and $E_{\bar{\theta}}$ is the target encoder. The bar over $\theta$ means this encoder uses a **different set of weights** — specifically, an exponentially moving average of the context encoder's weights. Critically, we apply **stop-gradient** to the target encoder: it does not receive gradients during backpropagation.

**Computationally:** we pass the masked patches through this second encoder to get their "ground truth" embeddings. If we have 6 masked patches, $s_y$ has shape $(6, 128)$.

### 3.3 Predictor

$$\hat{s}_y = f_\theta(s_x, c)$$

This is the heart of JEPA. The predictor $f_\theta$ takes the context embeddings $s_x$ and a conditioning signal $c$ (which tells the predictor **where** the masked patches are), and predicts what the target embeddings should be.

**Computationally:** the predictor is a small Transformer that receives the context embeddings plus learnable "mask tokens" positioned at the masked locations, and outputs predicted embeddings for the masked positions only.

### 3.4 The Loss Function

$$\mathcal{L} = \| \hat{s}_y - \text{sg}(s_y) \|^2$$

This is a simple **Mean Squared Error (MSE)** between the predicted embeddings $\hat{s}_y$ and the actual target embeddings $s_y$. The $\text{sg}(\cdot)$ denotes **stop-gradient** — we do not backpropagate through the target encoder.

**Computationally:** for each masked patch, we compute the squared difference between the predicted embedding vector and the actual embedding vector, then average across all masked patches.

Let us plug in some simple numbers to see how this works.

Suppose we have 2 masked patches with embedding dimension 3:

- Predicted embeddings: $\hat{s}_y = [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]$
- Target embeddings: $s_y = [[1.1, 1.9, 3.2], [3.8, 5.1, 5.9]]$

The loss for each patch:
- Patch 1: $(1.0 - 1.1)^2 + (2.0 - 1.9)^2 + (3.0 - 3.2)^2 = 0.01 + 0.01 + 0.04 = 0.06$
- Patch 2: $(4.0 - 3.8)^2 + (5.0 - 5.1)^2 + (6.0 - 5.9)^2 = 0.04 + 0.01 + 0.01 = 0.06$

Average loss: $(0.06 + 0.06) / 2 = 0.06$

This tells us our predictor is fairly close to the targets — a loss of 0.06 means the predicted embeddings are quite accurate. This is exactly what we want: as training progresses, this loss should decrease, meaning our predictions in abstract space get better and better.

### 3.5 EMA Update — Preventing Representation Collapse

$$\bar{\theta} \leftarrow \tau \bar{\theta} + (1 - \tau)\theta$$

Here, $\tau$ is the EMA decay rate (typically 0.996 to 0.999), $\bar{\theta}$ are the target encoder weights, and $\theta$ are the context encoder weights.

**Why do we need this?** Without the EMA + stop-gradient mechanism, the model could find a trivial solution: map every input to the same constant embedding (say, all zeros). Then the predictor's job is trivially easy — just predict zeros. The loss would be zero, but the representations would be completely useless. This is called **representation collapse**.

The EMA update prevents this by making the target encoder a **slowly evolving** version of the context encoder. Since the target does not receive gradients, it cannot collapse to a trivial solution. And since it tracks the context encoder slowly, it provides a stable, meaningful target.

Let us plug in some numbers. Suppose $\tau = 0.99$:

- Current target encoder weight: $\bar{\theta} = 5.0$
- Current context encoder weight: $\theta = 5.5$
- After EMA update: $\bar{\theta} \leftarrow 0.99 \times 5.0 + 0.01 \times 5.5 = 4.95 + 0.055 = 5.005$

Notice how the target encoder weight barely changed — it moved from 5.0 to 5.005. This is the "slow" tracking that keeps the target stable. Over hundreds of training steps, the target gradually catches up to the context encoder, providing a smooth and stable learning signal.

## 4. Let Us Build It — Component by Component

Now let us implement I-JEPA from scratch. We will build each component independently, test it, and then combine everything.

### 4.1 Load and Explore CIFAR-10

In [ ]:
# --- Load CIFAR-10 ---
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2470, 0.2399, 0.2010))
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True,
    download=True, transform=transform
)
testset = torchvision.datasets.CIFAR10(
    root='./data', train=False,
    download=True, transform=transform
)

CLASSES = trainset.classes
print(f"Training samples: {len(trainset)}")
print(f"Test samples: {len(testset)}")
print(f"Classes: {CLASSES}")
print(f"Image shape: {trainset[0][0].shape}")

In [ ]:
# 📊 Visualize some CIFAR-10 images
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
# Unnormalize for display
mean = torch.tensor([0.4914, 0.4822, 0.4465]).view(3, 1, 1)
std = torch.tensor([0.2470, 0.2399, 0.2010]).view(3, 1, 1)

for i, ax in enumerate(axes.flat):
    img, label = trainset[i]
    img_display = (img * std + mean).clamp(0, 1)
    ax.imshow(img_display.permute(1, 2, 0).numpy())
    ax.set_title(CLASSES[label], fontsize=10)
    ax.axis('off')

plt.suptitle("CIFAR-10 Sample Images", fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### 4.2 Patch Embedding — Splitting Images into Patches

The first step in I-JEPA is to split each image into a grid of patches and project each patch into an embedding space.

For CIFAR-10 (32x32 images), we will use **4x4 patches**, giving us a grid of **8x8 = 64 patches**, where each patch is 4x4 pixels with 3 color channels.

Think of it like cutting a photograph into 64 small squares. Each square gets converted into a single embedding vector.

In [ ]:
class PatchEmbedding(nn.Module):
    """Split image into patches and project to embedding dim."""

    def __init__(self, img_size=32, patch_size=4,
                 in_channels=3, embed_dim=128):
        super().__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = (img_size // patch_size) ** 2  # 64
        self.grid_size = img_size // patch_size            # 8

        # Conv2d with kernel=patch_size and stride=patch_size
        # acts as a learnable linear projection of each patch
        self.projection = nn.Conv2d(
            in_channels, embed_dim,
            kernel_size=patch_size, stride=patch_size
        )
        # Learnable position embeddings
        self.pos_embed = nn.Parameter(
            torch.randn(1, self.num_patches, embed_dim) * 0.02
        )

    def forward(self, x):
        """x: (B, C, H, W) -> (B, num_patches, embed_dim)"""
        B = x.shape[0]
        # Project patches: (B, embed_dim, grid, grid)
        x = self.projection(x)
        # Flatten spatial dims: (B, embed_dim, num_patches)
        x = x.flatten(2)
        # Transpose: (B, num_patches, embed_dim)
        x = x.transpose(1, 2)
        # Add positional embeddings
        x = x + self.pos_embed
        return x

In [ ]:
# 📊 Visualize patch splitting
sample_img, sample_label = trainset[0]
img_display = (sample_img * std + mean).clamp(0, 1)

fig, axes = plt.subplots(1, 2, figsize=(10, 4))

# Original image
axes[0].imshow(img_display.permute(1, 2, 0).numpy())
axes[0].set_title(f"Original: {CLASSES[sample_label]}", fontsize=12)
axes[0].axis('off')

# Image with patch grid overlay
axes[1].imshow(img_display.permute(1, 2, 0).numpy())
patch_size = 4
for i in range(0, 33, patch_size):
    axes[1].axhline(y=i-0.5, color='red', linewidth=0.8, alpha=0.7)
    axes[1].axvline(x=i-0.5, color='red', linewidth=0.8, alpha=0.7)
axes[1].set_title(f"8x8 Grid = 64 Patches (each {patch_size}x{patch_size})",
                  fontsize=12)
axes[1].axis('off')

plt.suptitle("Step 1: Split Image into Patches", fontsize=14,
             fontweight='bold')
plt.tight_layout()
plt.show()
print(f"Each 32x32 image -> {(32//patch_size)**2} patches "
      f"of size {patch_size}x{patch_size}")

### 4.3 Building the Transformer Encoder

Now we need the encoder — a simplified **Vision Transformer (ViT)**. This is the backbone that transforms patch embeddings into rich semantic representations.

Each Transformer block consists of:
1. **Multi-Head Self-Attention** — patches attend to each other
2. **Feed-Forward Network** — per-patch nonlinear transformation
3. **Layer Normalization** and **residual connections** for stable training

In [ ]:
class MultiHeadSelfAttention(nn.Module):
    """Multi-head self-attention for patch embeddings."""

    def __init__(self, embed_dim=128, num_heads=4):
        super().__init__()
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        self.scale = self.head_dim ** -0.5

        self.qkv = nn.Linear(embed_dim, 3 * embed_dim)
        self.proj = nn.Linear(embed_dim, embed_dim)

    def forward(self, x):
        """x: (B, N, D) -> (B, N, D)"""
        B, N, D = x.shape
        # Compute Q, K, V in one projection
        qkv = self.qkv(x).reshape(
            B, N, 3, self.num_heads, self.head_dim
        ).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]

        # Scaled dot-product attention
        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)

        # Combine heads
        x = (attn @ v).transpose(1, 2).reshape(B, N, D)
        return self.proj(x)

In [ ]:
class TransformerBlock(nn.Module):
    """Single Transformer encoder block."""

    def __init__(self, embed_dim=128, num_heads=4,
                 mlp_ratio=4.0):
        super().__init__()
        self.norm1 = nn.LayerNorm(embed_dim)
        self.attn = MultiHeadSelfAttention(embed_dim, num_heads)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, int(embed_dim * mlp_ratio)),
            nn.GELU(),
            nn.Linear(int(embed_dim * mlp_ratio), embed_dim),
        )

    def forward(self, x):
        """x: (B, N, D) -> (B, N, D)"""
        x = x + self.attn(self.norm1(x))   # Residual + attention
        x = x + self.mlp(self.norm2(x))    # Residual + FFN
        return x


class ViTEncoder(nn.Module):
    """Simplified Vision Transformer encoder."""

    def __init__(self, embed_dim=128, depth=4,
                 num_heads=4, mlp_ratio=4.0):
        super().__init__()
        self.blocks = nn.ModuleList([
            TransformerBlock(embed_dim, num_heads, mlp_ratio)
            for _ in range(depth)
        ])
        self.norm = nn.LayerNorm(embed_dim)

    def forward(self, x):
        """x: (B, N, D) -> (B, N, D)"""
        for block in self.blocks:
            x = block(x)
        return self.norm(x)

Let us verify that our encoder works correctly. We will pass a batch of patch embeddings through it and check the output shape.

In [ ]:
# Quick test of patch embedding + encoder
test_batch = torch.randn(4, 3, 32, 32)  # 4 images
patch_embed = PatchEmbedding(embed_dim=128)
encoder = ViTEncoder(embed_dim=128, depth=4)

patches = patch_embed(test_batch)
print(f"Patch embeddings shape: {patches.shape}")
# Expected: (4, 64, 128) = (batch, num_patches, embed_dim)

encoded = encoder(patches)
print(f"Encoder output shape: {encoded.shape}")
# Expected: (4, 64, 128)

print("✅ Patch embedding and encoder work correctly!")

### 4.4 The Target Encoder — Same Architecture, EMA Weights

The target encoder has the **exact same architecture** as the context encoder, but its weights are not updated by gradients. Instead, they are updated using EMA (we will implement this in Section 4.7).

At initialization, we simply copy the context encoder's weights.

In [ ]:
def create_target_encoder(context_encoder):
    """Create target encoder by copying context encoder weights.

    The target encoder has identical architecture but its
    parameters will be updated via EMA, not gradients.
    """
    import copy
    target_encoder = copy.deepcopy(context_encoder)
    # Freeze: target encoder does not receive gradients
    for param in target_encoder.parameters():
        param.requires_grad = False
    return target_encoder

# Test it
target_enc = create_target_encoder(encoder)
print(f"Target encoder parameters frozen: "
      f"{all(not p.requires_grad for p in target_enc.parameters())}")
print("✅ Target encoder created successfully!")

### 4.5 The Predictor — Predicting Target Embeddings

Now let us build the predictor. This is a **smaller Transformer** that takes the context embeddings (visible patch embeddings) plus learnable **mask tokens** (placed at the masked positions), and predicts the target embeddings.

The mask tokens act as "queries" — they say "I am at position (3, 5) in the grid, and I need you to fill in my embedding based on what you can see from the visible patches."

In [ ]:
class Predictor(nn.Module):
    """Predict target embeddings from context embeddings.

    Takes context (visible) embeddings and mask token positions,
    outputs predictions for the masked positions only.
    """

    def __init__(self, embed_dim=128, predictor_dim=64,
                 depth=2, num_heads=4):
        super().__init__()
        # Project from encoder dim to predictor dim
        self.input_proj = nn.Linear(embed_dim, predictor_dim)
        # Learnable mask token
        self.mask_token = nn.Parameter(
            torch.randn(1, 1, predictor_dim) * 0.02
        )
        # Positional embedding for predictor
        self.pos_embed = nn.Parameter(
            torch.randn(1, 64, predictor_dim) * 0.02  # 64 patches
        )
        # Transformer blocks
        self.blocks = nn.ModuleList([
            TransformerBlock(predictor_dim, num_heads, mlp_ratio=4.0)
            for _ in range(depth)
        ])
        self.norm = nn.LayerNorm(predictor_dim)
        # Project back to encoder dim for loss computation
        self.output_proj = nn.Linear(predictor_dim, embed_dim)

    def forward(self, context_emb, context_idx, target_idx):
        """
        Args:
            context_emb: (B, N_ctx, embed_dim) visible embeddings
            context_idx: (B, N_ctx) indices of visible patches
            target_idx:  (B, N_tgt) indices of masked patches
        Returns:
            predicted: (B, N_tgt, embed_dim) predicted embeddings
        """
        B = context_emb.shape[0]
        N_tgt = target_idx.shape[1]

        # Project context to predictor dimension
        x_ctx = self.input_proj(context_emb)

        # Create mask tokens for target positions
        mask_tokens = self.mask_token.expand(B, N_tgt, -1)

        # Add positional embeddings
        ctx_pos = self._gather_pos(context_idx)
        tgt_pos = self._gather_pos(target_idx)
        x_ctx = x_ctx + ctx_pos
        mask_tokens = mask_tokens + tgt_pos

        # Concatenate context and mask tokens
        x = torch.cat([x_ctx, mask_tokens], dim=1)

        # Transformer blocks
        for block in self.blocks:
            x = block(x)
        x = self.norm(x)

        # Extract only the mask token outputs
        predicted = x[:, -N_tgt:]

        # Project back to encoder embedding dim
        return self.output_proj(predicted)

    def _gather_pos(self, indices):
        """Gather positional embeddings for given indices."""
        B = indices.shape[0]
        pos = self.pos_embed.expand(B, -1, -1)
        indices_expanded = indices.unsqueeze(-1).expand(
            -1, -1, pos.shape[-1]
        )
        return torch.gather(pos, 1, indices_expanded)

In [ ]:
# Test the predictor
predictor = Predictor(embed_dim=128, predictor_dim=64, depth=2)

# Simulate: 40 visible patches, 24 masked patches
B = 4
context_emb = torch.randn(B, 40, 128)
context_idx = torch.arange(40).unsqueeze(0).expand(B, -1)
target_idx = torch.arange(40, 64).unsqueeze(0).expand(B, -1)

predicted = predictor(context_emb, context_idx, target_idx)
print(f"Predicted embeddings shape: {predicted.shape}")
# Expected: (4, 24, 128)
print("✅ Predictor works correctly!")

### 4.6 Multi-Block Masking Strategy

I-JEPA does not mask random individual patches. Instead, it masks **contiguous rectangular blocks** — this forces the model to predict larger semantic regions rather than trivially interpolating from neighbors.

The masking strategy generates rectangular blocks covering roughly **40-60%** of the patches.

In [ ]:
def create_multi_block_mask(grid_size=8, num_blocks=4,
                            min_block_size=2, max_block_size=4):
    """Create a mask by selecting contiguous rectangular blocks.

    Returns:
        mask: (grid_size * grid_size,) boolean tensor
              True = masked (target), False = visible (context)
    """
    mask = torch.zeros(grid_size, grid_size, dtype=torch.bool)

    for _ in range(num_blocks):
        # Random block size
        bh = torch.randint(min_block_size, max_block_size + 1, (1,)).item()
        bw = torch.randint(min_block_size, max_block_size + 1, (1,)).item()
        # Random top-left corner
        top = torch.randint(0, grid_size - bh + 1, (1,)).item()
        left = torch.randint(0, grid_size - bw + 1, (1,)).item()
        # Apply block mask
        mask[top:top+bh, left:left+bw] = True

    return mask.flatten()  # (64,)


def get_context_target_indices(mask):
    """Split patch indices into context (visible) and target (masked)."""
    context_idx = (~mask).nonzero(as_tuple=True)[0]
    target_idx = mask.nonzero(as_tuple=True)[0]
    return context_idx, target_idx

In [ ]:
# 📊 Visualize the masking strategy
fig, axes = plt.subplots(2, 4, figsize=(14, 7))

for i in range(8):
    row, col = i // 4, i % 4
    mask = create_multi_block_mask(grid_size=8)
    mask_2d = mask.reshape(8, 8).numpy()

    ctx_idx, tgt_idx = get_context_target_indices(mask)
    mask_ratio = len(tgt_idx) / 64 * 100

    axes[row, col].imshow(mask_2d, cmap='RdYlGn_r',
                          vmin=0, vmax=1)
    axes[row, col].set_title(f"Mask ratio: {mask_ratio:.0f}%",
                             fontsize=10)
    # Add grid lines
    for g in range(9):
        axes[row, col].axhline(y=g-0.5, color='gray',
                               linewidth=0.5, alpha=0.5)
        axes[row, col].axvline(x=g-0.5, color='gray',
                               linewidth=0.5, alpha=0.5)
    axes[row, col].axis('off')

plt.suptitle("Multi-Block Masking Examples\n"
             "(Red = Masked/Target, Green = Visible/Context)",
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

Now let us see what a masked image looks like visually. This helps us understand what the model "sees" versus what it needs to predict.

In [ ]:
# 📊 Show masked vs visible patches on a real image
sample_img, sample_label = trainset[7]
img_display = (sample_img * std + mean).clamp(0, 1).permute(1, 2, 0).numpy()

mask = create_multi_block_mask(grid_size=8)
ctx_idx, tgt_idx = get_context_target_indices(mask)

# Create masked version of the image
masked_img = img_display.copy()
mask_2d = mask.reshape(8, 8).numpy()
for r in range(8):
    for c in range(8):
        if mask_2d[r, c]:
            masked_img[r*4:(r+1)*4, c*4:(c+1)*4] = 0.5  # Gray out

fig, axes = plt.subplots(1, 3, figsize=(12, 4))
axes[0].imshow(img_display)
axes[0].set_title(f"Original: {CLASSES[sample_label]}", fontsize=12)
axes[0].axis('off')

axes[1].imshow(masked_img)
axes[1].set_title(f"Visible to Context Encoder\n"
                  f"({len(ctx_idx)} patches)", fontsize=12)
axes[1].axis('off')

# Show only masked region
target_only = np.ones_like(img_display) * 0.8
for r in range(8):
    for c in range(8):
        if mask_2d[r, c]:
            target_only[r*4:(r+1)*4, c*4:(c+1)*4] = \
                img_display[r*4:(r+1)*4, c*4:(c+1)*4]
axes[2].imshow(target_only)
axes[2].set_title(f"Target to Predict\n"
                  f"({len(tgt_idx)} patches)", fontsize=12)
axes[2].axis('off')

plt.suptitle("JEPA Masking: Context vs Target",
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### 4.7 EMA Update for Target Encoder

The final component is the EMA update rule. After every training step, we slowly update the target encoder's weights to track the context encoder.

In [ ]:
@torch.no_grad()
def ema_update(context_encoder, target_encoder, tau=0.996):
    """Update target encoder weights via exponential moving average.

    target_params <- tau * target_params + (1 - tau) * context_params

    Args:
        context_encoder: encoder being trained with gradients
        target_encoder: encoder updated via EMA (no gradients)
        tau: decay rate (higher = slower update, more stable)
    """
    for ctx_param, tgt_param in zip(
        context_encoder.parameters(),
        target_encoder.parameters()
    ):
        tgt_param.data.mul_(tau).add_(
            ctx_param.data, alpha=1.0 - tau
        )

In [ ]:
# Verify EMA update works
import copy
enc_a = nn.Linear(10, 10)
enc_b = copy.deepcopy(enc_a)

# Modify enc_a (simulate a gradient step)
with torch.no_grad():
    enc_a.weight.add_(torch.randn_like(enc_a.weight) * 0.1)

# Check they diverged
diff_before = (enc_a.weight - enc_b.weight).abs().mean().item()

# Apply EMA
ema_update(enc_a, enc_b, tau=0.99)

# Check they got closer
diff_after = (enc_a.weight - enc_b.weight).abs().mean().item()
print(f"Weight difference before EMA: {diff_before:.4f}")
print(f"Weight difference after EMA:  {diff_after:.4f}")
print(f"Target moved {((diff_before - diff_after)/diff_before)*100:.1f}%"
      f" closer to context")
print("✅ EMA update works correctly!")

## 5. 🔧 Your Turn — Implement Key Components

Now it is your turn. We have built all the pieces; here are two critical functions for you to implement.

### TODO 1: Implement Multi-Block Masking

Complete the function below that creates a multi-block mask ensuring the mask ratio stays within 40-60% of total patches.

In [ ]:
def create_balanced_mask(grid_size=8, target_ratio=0.5,
                         tolerance=0.1):
    """Create a multi-block mask with a controlled mask ratio.

    Generate rectangular blocks until the mask ratio falls
    within [target_ratio - tolerance, target_ratio + tolerance].

    Args:
        grid_size: number of patches per side (8 for our setup)
        target_ratio: desired fraction of patches to mask
        tolerance: acceptable deviation from target_ratio

    Returns:
        mask: (grid_size * grid_size,) boolean tensor

    Hints:
        - Start with an empty (all False) mask
        - Repeatedly add random rectangular blocks
        - Check the ratio after each block
        - Stop when within the target range
        - If you overshoot, start over
    """
    total_patches = grid_size * grid_size
    min_ratio = target_ratio - tolerance
    max_ratio = target_ratio + tolerance

    # ============ TODO ============
    # Step 1: Initialize an empty mask (grid_size x grid_size, all False)
    # Step 2: Loop — add random rectangular blocks (size 2-4)
    # Step 3: After each block, compute current mask ratio
    # Step 4: If ratio is in [min_ratio, max_ratio], return mask
    # Step 5: If ratio exceeds max_ratio, reset and try again
    # ==============================

    mask = None  # YOUR CODE HERE

    return mask.flatten()

In [ ]:
# ✅ Verification: Test your create_balanced_mask implementation
print("Testing create_balanced_mask()...")
passed = True
for trial in range(20):
    mask = create_balanced_mask(grid_size=8)
    ratio = mask.float().mean().item()
    if not (0.4 <= ratio <= 0.6):
        print(f"❌ Trial {trial}: mask ratio {ratio:.2f} "
              f"is outside [0.4, 0.6]")
        passed = False

    # Check that mask is a boolean tensor of correct size
    if mask.shape != (64,):
        print(f"❌ Trial {trial}: expected shape (64,), "
              f"got {mask.shape}")
        passed = False

if passed:
    print("✅ All 20 trials passed! Mask ratios are within "
          "[40%, 60%] and shapes are correct.")

### TODO 2: Implement the JEPA Loss Function

Complete the loss function that computes MSE between predicted and target embeddings, with stop-gradient on the target.

In [ ]:
def jepa_loss(predicted_emb, target_emb):
    """Compute JEPA loss: MSE with stop-gradient on target.

    Loss = mean(|| predicted - sg(target) ||^2)

    Args:
        predicted_emb: (B, N_tgt, D) predicted target embeddings
        target_emb: (B, N_tgt, D) actual target embeddings

    Returns:
        loss: scalar tensor

    Hints:
        - Use target_emb.detach() to apply stop-gradient
        - Compute squared L2 distance per embedding vector
        - Average over all patches and batch elements
        - F.mse_loss does exactly this
    """
    # ============ TODO ============
    # Step 1: Detach target embeddings (stop-gradient)
    # Step 2: Compute MSE between predicted and detached target
    # ==============================

    loss = None  # YOUR CODE HERE

    return loss

In [ ]:
# ✅ Verification: Test your jepa_loss implementation
pred = torch.tensor([[[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]])
targ = torch.tensor([[[1.1, 1.9, 3.2], [3.8, 5.1, 5.9]]])

loss = jepa_loss(pred, targ)
expected = 0.06  # We computed this by hand in Section 3.4!

print(f"Your loss: {loss.item():.4f}")
print(f"Expected:  {expected:.4f}")

if abs(loss.item() - expected) < 1e-3:
    print("✅ Correct! Your loss matches our hand calculation.")
else:
    print("❌ Loss does not match. Check your implementation.")

# Verify stop-gradient: target should not require grad
targ_grad = torch.randn(2, 3, 4, requires_grad=True)
pred_grad = torch.randn(2, 3, 4, requires_grad=True)
l = jepa_loss(pred_grad, targ_grad)
l.backward()
assert pred_grad.grad is not None, "❌ Predicted should have gradient"
assert targ_grad.grad is None, "❌ Target should NOT have gradient"
print("✅ Stop-gradient verified: gradients flow to predictor "
      "but not to target encoder.")

## 6. Putting It All Together — The Full I-JEPA Model

Now let us assemble all our components into a single, clean I-JEPA model.

In [ ]:
class IJEPA(nn.Module):
    """Image-based Joint Embedding Predictive Architecture.

    Components:
        - Patch embedding: splits image into patches
        - Context encoder: ViT that encodes visible patches
        - Target encoder: EMA copy that encodes masked patches
        - Predictor: small transformer that predicts target embeddings
    """

    def __init__(self, img_size=32, patch_size=4,
                 embed_dim=128, encoder_depth=4,
                 predictor_dim=64, predictor_depth=2,
                 num_heads=4):
        super().__init__()
        self.patch_embed = PatchEmbedding(
            img_size, patch_size, 3, embed_dim
        )
        self.context_encoder = ViTEncoder(
            embed_dim, encoder_depth, num_heads
        )
        self.target_encoder = create_target_encoder(
            self.context_encoder
        )
        self.predictor = Predictor(
            embed_dim, predictor_dim,
            predictor_depth, num_heads
        )
        self.embed_dim = embed_dim
        self.num_patches = self.patch_embed.num_patches

    def forward(self, images, context_idx, target_idx):
        """
        Args:
            images: (B, 3, H, W) input images
            context_idx: (B, N_ctx) visible patch indices
            target_idx: (B, N_tgt) masked patch indices
        Returns:
            predicted: (B, N_tgt, embed_dim)
            target: (B, N_tgt, embed_dim)
        """
        # Get all patch embeddings
        all_patches = self.patch_embed(images)

        # Gather context patches and encode
        ctx_patches = self._gather(all_patches, context_idx)
        ctx_encoded = self.context_encoder(ctx_patches)

        # Gather target patches and encode (no gradient!)
        with torch.no_grad():
            tgt_patches = self._gather(all_patches, target_idx)
            tgt_encoded = self.target_encoder(tgt_patches)

        # Predict target embeddings from context
        predicted = self.predictor(
            ctx_encoded, context_idx, target_idx
        )

        return predicted, tgt_encoded

    def _gather(self, patches, indices):
        """Gather specific patches by index."""
        B, N, D = patches.shape
        indices_expanded = indices.unsqueeze(-1).expand(-1, -1, D)
        return torch.gather(patches, 1, indices_expanded)

    def update_target_encoder(self, tau=0.996):
        """Update target encoder via EMA."""
        ema_update(
            self.context_encoder,
            self.target_encoder, tau
        )

In [ ]:
# Test the full model
model = IJEPA(
    img_size=32, patch_size=4, embed_dim=128,
    encoder_depth=4, predictor_dim=64,
    predictor_depth=2, num_heads=4
).to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable = sum(p.numel() for p in model.parameters()
                if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable:,}")
print(f"Target encoder (frozen): {total_params - trainable:,}")

## 7. Training and Results

### 7.1 Training Loop

Now let us train our I-JEPA on CIFAR-10. We will train for 15 epochs — this should complete in about 3-4 minutes on a T4 GPU.

In [ ]:
# --- Training Configuration ---
BATCH_SIZE = 256
LEARNING_RATE = 1.5e-3
NUM_EPOCHS = 15
EMA_TAU = 0.996
GRID_SIZE = 8

train_loader = torch.utils.data.DataLoader(
    trainset, batch_size=BATCH_SIZE,
    shuffle=True, num_workers=2,
    pin_memory=True, drop_last=True
)

model = IJEPA(
    img_size=32, patch_size=4, embed_dim=128,
    encoder_depth=4, predictor_dim=64,
    predictor_depth=2, num_heads=4
).to(device)

optimizer = torch.optim.AdamW(
    [p for p in model.parameters() if p.requires_grad],
    lr=LEARNING_RATE, weight_decay=0.05
)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=NUM_EPOCHS
)

In [ ]:
# --- Training Loop ---
loss_history = []

for epoch in range(NUM_EPOCHS):
    model.train()
    epoch_losses = []

    for batch_idx, (images, _) in enumerate(train_loader):
        images = images.to(device)
        B = images.shape[0]

        # Generate masks for this batch
        masks = torch.stack(
            [create_multi_block_mask(GRID_SIZE) for _ in range(B)]
        )
        ctx_indices = []
        tgt_indices = []
        for m in masks:
            ci, ti = get_context_target_indices(m)
            ctx_indices.append(ci)
            tgt_indices.append(ti)

        # Pad to same length within batch
        max_ctx = max(c.shape[0] for c in ctx_indices)
        max_tgt = max(t.shape[0] for t in tgt_indices)

        ctx_batch = torch.zeros(B, max_ctx, dtype=torch.long)
        tgt_batch = torch.zeros(B, max_tgt, dtype=torch.long)

        for i in range(B):
            ctx_batch[i, :ctx_indices[i].shape[0]] = ctx_indices[i]
            tgt_batch[i, :tgt_indices[i].shape[0]] = tgt_indices[i]

        ctx_batch = ctx_batch.to(device)
        tgt_batch = tgt_batch.to(device)

        # Forward pass
        predicted, target = model(images, ctx_batch, tgt_batch)

        # Compute loss
        loss = F.mse_loss(predicted, target.detach())

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        # Update target encoder
        model.update_target_encoder(tau=EMA_TAU)

        epoch_losses.append(loss.item())

    scheduler.step()
    avg_loss = np.mean(epoch_losses)
    loss_history.append(avg_loss)

    if (epoch + 1) % 3 == 0 or epoch == 0:
        print(f"Epoch [{epoch+1:2d}/{NUM_EPOCHS}] | "
              f"Loss: {avg_loss:.4f} | "
              f"LR: {scheduler.get_last_lr()[0]:.6f}")

print(f"\nTraining complete! Final loss: {loss_history[-1]:.4f}")

### 7.2 Training Loss Curve

Let us visualize how the training loss evolved. A steadily decreasing loss means our predictor is learning to predict target embeddings more and more accurately.

In [ ]:
# 📊 Plot training loss
plt.figure(figsize=(10, 5))
plt.plot(range(1, NUM_EPOCHS + 1), loss_history,
         'b-o', linewidth=2, markersize=6)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('MSE Loss', fontsize=12)
plt.title('I-JEPA Training Loss on CIFAR-10', fontsize=14,
          fontweight='bold')
plt.grid(True, alpha=0.3)
plt.xticks(range(1, NUM_EPOCHS + 1))
plt.tight_layout()
plt.show()

reduction = (loss_history[0] - loss_history[-1]) / loss_history[0]
print(f"Loss reduced by {reduction*100:.1f}% over {NUM_EPOCHS} "
      f"epochs — the model is learning to predict in abstract "
      f"space!")

### 7.3 📊 How Predictions Improve Over Training

Let us visualize how the predicted embeddings get closer to the target embeddings as training progresses. We will use cosine similarity as our metric.

In [ ]:
# 📊 Measure prediction quality
model.eval()
sample_images, _ = next(iter(train_loader))
sample_images = sample_images[:16].to(device)

with torch.no_grad():
    B = sample_images.shape[0]
    masks = torch.stack(
        [create_multi_block_mask(GRID_SIZE) for _ in range(B)]
    )
    ctx_list, tgt_list = [], []
    for m in masks:
        ci, ti = get_context_target_indices(m)
        ctx_list.append(ci)
        tgt_list.append(ti)

    max_ctx = max(c.shape[0] for c in ctx_list)
    max_tgt = max(t.shape[0] for t in tgt_list)

    ctx_b = torch.zeros(B, max_ctx, dtype=torch.long)
    tgt_b = torch.zeros(B, max_tgt, dtype=torch.long)
    for i in range(B):
        ctx_b[i, :ctx_list[i].shape[0]] = ctx_list[i]
        tgt_b[i, :tgt_list[i].shape[0]] = tgt_list[i]

    ctx_b, tgt_b = ctx_b.to(device), tgt_b.to(device)
    pred, targ = model(sample_images, ctx_b, tgt_b)

    # Cosine similarity per patch
    cos_sim = F.cosine_similarity(pred, targ, dim=-1)
    avg_sim = cos_sim.mean().item()

print(f"Average cosine similarity between predicted "
      f"and target embeddings: {avg_sim:.4f}")
print(f"(1.0 = perfect prediction, 0.0 = orthogonal)")

### 7.4 Linear Probing — Evaluating Representation Quality

The real test of self-supervised learning: are the learned representations **useful**? We freeze the encoder and train a simple linear classifier on top. If the representations are good, even a linear classifier should achieve decent accuracy.

This is called **linear probing** — it measures how much useful information the encoder has captured about the data.

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler

@torch.no_grad()
def extract_features(model, dataloader, device, max_samples=10000):
    """Extract features from the context encoder."""
    model.eval()
    features, labels = [], []
    count = 0

    for images, targets in dataloader:
        if count >= max_samples:
            break
        images = images.to(device)

        # Encode all patches (no masking for evaluation)
        patches = model.patch_embed(images)
        encoded = model.context_encoder(patches)

        # Global average pooling over patches
        pooled = encoded.mean(dim=1)  # (B, embed_dim)

        features.append(pooled.cpu().numpy())
        labels.append(targets.numpy())
        count += images.shape[0]

    features = np.concatenate(features)[:max_samples]
    labels = np.concatenate(labels)[:max_samples]
    return features, labels

In [ ]:
# Extract features from trained I-JEPA encoder
eval_loader = torch.utils.data.DataLoader(
    trainset, batch_size=256, shuffle=False, num_workers=2
)
test_loader = torch.utils.data.DataLoader(
    testset, batch_size=256, shuffle=False, num_workers=2
)

print("Extracting JEPA features from training set...")
train_feats, train_labels = extract_features(
    model, eval_loader, device, max_samples=10000
)
print("Extracting JEPA features from test set...")
test_feats, test_labels = extract_features(
    model, test_loader, device, max_samples=5000
)

# Standardize features
scaler = StandardScaler()
train_feats_scaled = scaler.fit_transform(train_feats)
test_feats_scaled = scaler.transform(test_feats)

print(f"Train features shape: {train_feats.shape}")
print(f"Test features shape: {test_feats.shape}")

In [ ]:
# Train linear classifier on JEPA features
print("Training linear classifier on JEPA features...")
clf_jepa = SGDClassifier(
    loss='hinge', max_iter=1000,
    random_state=42, tol=1e-3
)
clf_jepa.fit(train_feats_scaled, train_labels)
jepa_acc = clf_jepa.score(test_feats_scaled, test_labels)
print(f"JEPA Linear Probe Accuracy: {jepa_acc*100:.1f}%")

# Compare with random encoder (untrained)
import copy
random_model = IJEPA(
    img_size=32, patch_size=4, embed_dim=128,
    encoder_depth=4, predictor_dim=64,
    predictor_depth=2, num_heads=4
).to(device)

print("\nExtracting random encoder features...")
rand_train, _ = extract_features(
    random_model, eval_loader, device, max_samples=10000
)
rand_test, _ = extract_features(
    random_model, test_loader, device, max_samples=5000
)

scaler_rand = StandardScaler()
rand_train_scaled = scaler_rand.fit_transform(rand_train)
rand_test_scaled = scaler_rand.transform(rand_test)

clf_rand = SGDClassifier(
    loss='hinge', max_iter=1000,
    random_state=42, tol=1e-3
)
clf_rand.fit(rand_train_scaled, train_labels)
rand_acc = clf_rand.score(rand_test_scaled, test_labels)
print(f"Random Encoder Linear Probe Accuracy: {rand_acc*100:.1f}%")

In [ ]:
# 📊 Compare accuracies
fig, ax = plt.subplots(figsize=(8, 5))
methods = ['Random Encoder', 'I-JEPA (Ours)']
accs = [rand_acc * 100, jepa_acc * 100]
colors = ['#d9534f', '#5cb85c']
bars = ax.bar(methods, accs, color=colors, width=0.5,
              edgecolor='black', linewidth=1.2)

for bar, acc in zip(bars, accs):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
            f'{acc:.1f}%', ha='center', fontsize=14,
            fontweight='bold')

ax.set_ylabel('Linear Probe Accuracy (%)', fontsize=12)
ax.set_title('Representation Quality: Random vs I-JEPA',
             fontsize=14, fontweight='bold')
ax.set_ylim(0, 100)
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

improvement = jepa_acc - rand_acc
print(f"\nI-JEPA representations are {improvement*100:.1f} "
      f"percentage points better than random!")
print("This confirms that JEPA learns meaningful, "
      "structured representations.")

## 8. 🎯 Final Output — Visualizing What JEPA Learned

This is the most satisfying part. Let us see **what** our model learned by visualizing the embedding space.

### 8.1 t-SNE: JEPA vs Random

We will project the 128-dimensional embeddings down to 2D using t-SNE and color them by class. If JEPA learned good representations, images of the same class should cluster together.

In [ ]:
from sklearn.manifold import TSNE

# t-SNE on JEPA features
print("Computing t-SNE for JEPA features (this takes ~30 sec)...")
tsne = TSNE(n_components=2, random_state=42,
            perplexity=30, n_iter=1000)
jepa_2d = tsne.fit_transform(test_feats_scaled[:2000])

# t-SNE on random features
print("Computing t-SNE for random features...")
tsne_rand = TSNE(n_components=2, random_state=42,
                 perplexity=30, n_iter=1000)
rand_2d = tsne_rand.fit_transform(rand_test_scaled[:2000])
labels_2d = test_labels[:2000]

print("Done!")

In [ ]:
# 📊 t-SNE visualization — the big reveal!
fig, axes = plt.subplots(1, 2, figsize=(16, 7))
cmap = plt.cm.get_cmap('tab10')

for i, cls_name in enumerate(CLASSES):
    mask_cls = labels_2d == i

    axes[0].scatter(
        rand_2d[mask_cls, 0], rand_2d[mask_cls, 1],
        c=[cmap(i)], label=cls_name, alpha=0.5, s=8
    )
    axes[1].scatter(
        jepa_2d[mask_cls, 0], jepa_2d[mask_cls, 1],
        c=[cmap(i)], label=cls_name, alpha=0.5, s=8
    )

axes[0].set_title('Random Encoder\n(No Structure)',
                   fontsize=14, fontweight='bold')
axes[0].legend(fontsize=8, markerscale=3, loc='upper right')
axes[0].set_xticks([])
axes[0].set_yticks([])

axes[1].set_title('I-JEPA Encoder\n(Clear Clustering!)',
                   fontsize=14, fontweight='bold')
axes[1].legend(fontsize=8, markerscale=3, loc='upper right')
axes[1].set_xticks([])
axes[1].set_yticks([])

plt.suptitle(
    't-SNE of CIFAR-10 Representations: '
    'Random vs I-JEPA',
    fontsize=16, fontweight='bold'
)
plt.tight_layout()
plt.show()

print("The I-JEPA plot should show clear clusters — images of "
      "the same class have similar embeddings.")
print("The random encoder plot should look like a scattered "
      "mess with no meaningful structure.")
print("\nThis is exactly what we want. JEPA learned "
      "powerful representations without generating "
      "a single pixel!")

### 8.2 Nearest-Neighbor Retrieval

Another way to verify the quality of learned representations: given a query image, find the images with the **most similar embeddings**. If the representations are good, the nearest neighbors should be visually and semantically similar.

In [ ]:
# Nearest-neighbor retrieval using JEPA embeddings
from sklearn.metrics.pairwise import cosine_similarity

# Use test features
sim_matrix = cosine_similarity(test_feats_scaled[:1000])

# Display query images and their nearest neighbors
fig, axes = plt.subplots(4, 6, figsize=(14, 10))
query_indices = [0, 42, 100, 250]

# Unnormalize helper
raw_transform = transforms.ToTensor()
raw_testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=False,
    transform=raw_transform
)

for row, qi in enumerate(query_indices):
    # Query image
    img_q, lbl_q = raw_testset[qi]
    axes[row, 0].imshow(img_q.permute(1, 2, 0).numpy())
    axes[row, 0].set_title(f"Query:\n{CLASSES[lbl_q]}",
                           fontsize=9, fontweight='bold',
                           color='blue')
    axes[row, 0].axis('off')
    # Add blue border
    for spine in axes[row, 0].spines.values():
        spine.set_edgecolor('blue')
        spine.set_linewidth(3)
        spine.set_visible(True)

    # Top-5 nearest neighbors (skip self)
    sims = sim_matrix[qi]
    nn_indices = np.argsort(sims)[::-1][1:6]

    for col, ni in enumerate(nn_indices):
        img_n, lbl_n = raw_testset[ni]
        axes[row, col+1].imshow(img_n.permute(1, 2, 0).numpy())
        match = "✓" if lbl_n == lbl_q else "✗"
        color = 'green' if lbl_n == lbl_q else 'red'
        axes[row, col+1].set_title(
            f"{match} {CLASSES[lbl_n]}\n"
            f"sim={sims[ni]:.2f}",
            fontsize=8, color=color
        )
        axes[row, col+1].axis('off')

plt.suptitle(
    "Nearest-Neighbor Retrieval Using JEPA Embeddings\n"
    "(Blue = Query, Green ✓ = Same Class, Red ✗ = Different)",
    fontsize=13, fontweight='bold'
)
plt.tight_layout()
plt.show()

### 8.3 Congratulations!

You have successfully built **I-JEPA from scratch** and demonstrated that it learns meaningful representations of images **without ever generating a single pixel**.

Let us recap what we accomplished:

| Component | What It Does |
|---|---|
| **Patch Embedding** | Splits 32x32 images into 64 patches, projects each to 128-d |
| **Context Encoder (ViT)** | Encodes visible patches into rich embeddings |
| **Target Encoder (EMA)** | Provides stable target embeddings via exponential moving average |
| **Predictor** | Predicts target embeddings from context embeddings |
| **Multi-Block Masking** | Masks contiguous regions to force semantic prediction |

And the results speak for themselves:
- The training loss decreased steadily, confirming the model learned to predict in abstract space
- Linear probing showed a large accuracy gap between JEPA and random features
- t-SNE revealed clear class-based clustering in the JEPA embedding space
- Nearest-neighbor retrieval returned semantically similar images

**This is exactly what we want.** JEPA focuses on the meaning, not the surface.

## 9. Reflection and Next Steps

### 🤔 Reflection Questions

Take a moment to think about these questions. They will deepen your understanding of JEPA and self-supervised learning.

1. **Why contiguous block masking?** We mask contiguous rectangular regions instead of random patches. What would happen if we masked random individual patches instead? Why might that be "too easy" for the predictor? *(Hint: think about what information neighboring patches provide.)*

2. **The role of the EMA target.** We said the EMA prevents representation collapse. But what would happen if we set $\tau = 0$ (target encoder instantly copies context encoder)? What about $\tau = 1$ (target encoder never updates)? Why is a value like 0.996 the sweet spot?

3. **JEPA vs Contrastive Learning.** Methods like SimCLR and DINO also learn representations without labels, but they use contrastive objectives (push different images apart, pull augmentations together). How is JEPA's predictive objective fundamentally different? What are the advantages?

### Optional Challenges

If you want to go further, try these:

**Challenge 1: V-JEPA Extension**
The temporal extension of JEPA (V-JEPA) predicts video frame embeddings from past frame embeddings. How would you modify our architecture to handle video? Sketch out the changes needed:
- How would patch embedding change for 3D data (spatial + temporal)?
- What would the masking strategy look like across frames?
- How would the predictor condition on both spatial and temporal context?

**Challenge 2: Improve Linear Probe Accuracy**
Try to push the linear probe accuracy higher by experimenting with:
- Increasing `embed_dim` from 128 to 256
- Training for more epochs (30-50)
- Using a different masking ratio (try 0.6 or 0.75)
- Adding data augmentation (random crops, color jitter)
- Using a deeper encoder (6-8 layers instead of 4)

Track how each change affects the final accuracy and training time.

### What Comes Next

In the next notebook, we will explore how these learned representations can be connected to **actions** — bridging the gap from perception to decision-making. We will see how models like **VLAs (Vision-Language-Action models)** combine visual understanding with language instructions to produce robot actions.

The key insight: JEPA gives us a powerful way to **understand** the world. But understanding is only half the story — we also need to **act** in the world. This brings us to the exciting intersection of representation learning and embodied AI.

See you in Notebook 5!

In [ ]:
#@title 💬 AI Teaching Assistant — Click ▶ to start
#@markdown This AI chatbot reads your notebook and can answer questions about any concept, code, or exercise.

import json as _json
import requests as _requests
from google.colab import output as _output
from IPython.display import display, HTML as _HTML, Markdown as _Markdown

# --- Read notebook content for context ---
def _get_notebook_context():
    try:
        from google.colab import _message
        nb = _message.blocking_request("get_ipynb", request="", timeout_sec=10)
        cells = nb.get("ipynb", {}).get("cells", [])
        parts = []
        for cell in cells:
            src = "".join(cell.get("source", []))
            tags = cell.get("metadata", {}).get("tags", [])
            if "chatbot" in tags:
                continue
            if src.strip():
                ct = cell.get("cell_type", "unknown")
                parts.append(f"[{ct.upper()}]\n{src}")
        return "\n\n---\n\n".join(parts)
    except Exception:
        return "Notebook content unavailable."

_NOTEBOOK_CONTEXT = _get_notebook_context()
_CHAT_HISTORY = []
_API_URL = "https://course-creator-brown.vercel.app/api/chat"

def _notebook_chat(question):
    global _CHAT_HISTORY
    try:
        resp = _requests.post(_API_URL, json={
            'question': question,
            'context': _NOTEBOOK_CONTEXT[:100000],
            'history': _CHAT_HISTORY[-10:],
        }, timeout=60)
        data = resp.json()
        answer = data.get('answer', 'Sorry, I could not generate a response.')
        _CHAT_HISTORY.append({'role': 'user', 'content': question})
        _CHAT_HISTORY.append({'role': 'assistant', 'content': answer})
        return answer
    except Exception as e:
        return f'Error connecting to teaching assistant: {str(e)}'

_output.register_callback('notebook_chat', _notebook_chat)

def ask(question):
    """Ask the AI teaching assistant a question about this notebook."""
    answer = _notebook_chat(question)
    display(_Markdown(answer))

print("\u2705 AI Teaching Assistant is ready!")
print("\U0001f4a1 Use the chat below, or call ask(\'your question\') in any cell.")

# --- Display chat widget ---
display(_HTML('''<style>
  .vc-wrap{font-family:-apple-system,BlinkMacSystemFont,'Segoe UI',Roboto,sans-serif;max-width:100%;border-radius:16px;overflow:hidden;box-shadow:0 4px 24px rgba(0,0,0,.12);background:#fff;border:1px solid #e5e7eb}
  .vc-hdr{background:linear-gradient(135deg,#667eea 0%,#764ba2 100%);color:#fff;padding:16px 20px;display:flex;align-items:center;gap:12px}
  .vc-avatar{width:42px;height:42px;background:rgba(255,255,255,.2);border-radius:50%;display:flex;align-items:center;justify-content:center;font-size:22px}
  .vc-hdr h3{font-size:16px;font-weight:600;margin:0}
  .vc-hdr p{font-size:12px;opacity:.85;margin:2px 0 0}
  .vc-msgs{height:420px;overflow-y:auto;padding:16px;background:#f8f9fb;display:flex;flex-direction:column;gap:10px}
  .vc-msg{display:flex;flex-direction:column;animation:vc-fade .25s ease}
  .vc-msg.user{align-items:flex-end}
  .vc-msg.bot{align-items:flex-start}
  .vc-bbl{max-width:85%;padding:10px 14px;border-radius:16px;font-size:14px;line-height:1.55;word-wrap:break-word}
  .vc-msg.user .vc-bbl{background:linear-gradient(135deg,#667eea 0%,#764ba2 100%);color:#fff;border-bottom-right-radius:4px}
  .vc-msg.bot .vc-bbl{background:#fff;color:#1a1a2e;border:1px solid #e8e8e8;border-bottom-left-radius:4px}
  .vc-bbl code{background:rgba(0,0,0,.07);padding:2px 6px;border-radius:4px;font-size:13px;font-family:'Fira Code',monospace}
  .vc-bbl pre{background:#1e1e2e;color:#cdd6f4;padding:12px;border-radius:8px;overflow-x:auto;margin:8px 0;font-size:13px}
  .vc-bbl pre code{background:none;padding:0;color:inherit}
  .vc-bbl h3,.vc-bbl h4{margin:10px 0 4px;font-size:15px}
  .vc-bbl ul,.vc-bbl ol{margin:4px 0;padding-left:20px}
  .vc-bbl li{margin:2px 0}
  .vc-chips{display:flex;flex-wrap:wrap;gap:8px;padding:0 16px 12px;background:#f8f9fb}
  .vc-chip{background:#fff;border:1px solid #d1d5db;border-radius:20px;padding:6px 14px;font-size:12px;cursor:pointer;transition:all .15s;color:#4b5563}
  .vc-chip:hover{border-color:#667eea;color:#667eea;background:#f0f0ff}
  .vc-input{display:flex;padding:12px 16px;background:#fff;border-top:1px solid #eee;gap:8px}
  .vc-input input{flex:1;padding:10px 16px;border:2px solid #e8e8e8;border-radius:24px;font-size:14px;outline:none;transition:border-color .2s}
  .vc-input input:focus{border-color:#667eea}
  .vc-input button{background:linear-gradient(135deg,#667eea 0%,#764ba2 100%);color:#fff;border:none;border-radius:50%;width:42px;height:42px;cursor:pointer;display:flex;align-items:center;justify-content:center;font-size:18px;transition:transform .1s}
  .vc-input button:hover{transform:scale(1.05)}
  .vc-input button:disabled{opacity:.5;cursor:not-allowed;transform:none}
  .vc-typing{display:flex;gap:5px;padding:4px 0}
  .vc-typing span{width:8px;height:8px;background:#667eea;border-radius:50%;animation:vc-bounce 1.4s infinite ease-in-out}
  .vc-typing span:nth-child(2){animation-delay:.2s}
  .vc-typing span:nth-child(3){animation-delay:.4s}
  @keyframes vc-bounce{0%,80%,100%{transform:scale(0)}40%{transform:scale(1)}}
  @keyframes vc-fade{from{opacity:0;transform:translateY(8px)}to{opacity:1;transform:translateY(0)}}
  .vc-note{text-align:center;font-size:11px;color:#9ca3af;padding:8px 16px 12px;background:#fff}
</style>
<div class="vc-wrap">
  <div class="vc-hdr">
    <div class="vc-avatar">&#129302;</div>
    <div>
      <h3>Vizuara Teaching Assistant</h3>
      <p>Ask me anything about this notebook</p>
    </div>
  </div>
  <div class="vc-msgs" id="vcMsgs">
    <div class="vc-msg bot">
      <div class="vc-bbl">&#128075; Hi! I've read through this entire notebook. Ask me about any concept, code block, or exercise &mdash; I'm here to help you learn!</div>
    </div>
  </div>
  <div class="vc-chips" id="vcChips">
    <span class="vc-chip" onclick="vcAsk(this.textContent)">Explain the main concept</span>
    <span class="vc-chip" onclick="vcAsk(this.textContent)">Help with the TODO exercise</span>
    <span class="vc-chip" onclick="vcAsk(this.textContent)">Summarize what I learned</span>
  </div>
  <div class="vc-input">
    <input type="text" id="vcIn" placeholder="Ask about concepts, code, exercises..." />
    <button id="vcSend" onclick="vcSendMsg()">&#10148;</button>
  </div>
  <div class="vc-note">AI-generated &middot; Verify important information &middot; <a href="#" onclick="vcClear();return false" style="color:#667eea">Clear chat</a></div>
</div>
<script>
(function(){
  var msgs=document.getElementById('vcMsgs'),inp=document.getElementById('vcIn'),
      btn=document.getElementById('vcSend'),chips=document.getElementById('vcChips');

  function esc(s){var d=document.createElement('div');d.textContent=s;return d.innerHTML}

  function md(t){
    return t
      .replace(/```(\w*)\n([\s\S]*?)```/g,function(_,l,c){return '<pre><code>'+esc(c)+'</code></pre>'})
      .replace(/`([^`]+)`/g,'<code>$1</code>')
      .replace(/\*\*([^*]+)\*\*/g,'<strong>$1</strong>')
      .replace(/\*([^*]+)\*/g,'<em>$1</em>')
      .replace(/^#### (.+)$/gm,'<h4>$1</h4>')
      .replace(/^### (.+)$/gm,'<h4>$1</h4>')
      .replace(/^## (.+)$/gm,'<h3>$1</h3>')
      .replace(/^\d+\. (.+)$/gm,'<li>$1</li>')
      .replace(/^- (.+)$/gm,'<li>$1</li>')
      .replace(/\n\n/g,'<br><br>')
      .replace(/\n/g,'<br>');
  }

  function addMsg(text,isUser){
    var m=document.createElement('div');m.className='vc-msg '+(isUser?'user':'bot');
    var b=document.createElement('div');b.className='vc-bbl';
    b.innerHTML=isUser?esc(text):md(text);
    m.appendChild(b);msgs.appendChild(m);msgs.scrollTop=msgs.scrollHeight;
  }

  function showTyping(){
    var m=document.createElement('div');m.className='vc-msg bot';m.id='vcTyping';
    m.innerHTML='<div class="vc-bbl"><div class="vc-typing"><span></span><span></span><span></span></div></div>';
    msgs.appendChild(m);msgs.scrollTop=msgs.scrollHeight;
  }

  function hideTyping(){var e=document.getElementById('vcTyping');if(e)e.remove()}

  window.vcSendMsg=function(){
    var q=inp.value.trim();if(!q)return;
    inp.value='';chips.style.display='none';
    addMsg(q,true);showTyping();btn.disabled=true;
    google.colab.kernel.invokeFunction('notebook_chat',[q],{})
      .then(function(r){
        hideTyping();
        var a=r.data['application/json'];
        addMsg(typeof a==='string'?a:JSON.stringify(a),false);
      })
      .catch(function(){
        hideTyping();
        addMsg('Sorry, I encountered an error. Please check your internet connection and try again.',false);
      })
      .finally(function(){btn.disabled=false;inp.focus()});
  };

  window.vcAsk=function(q){inp.value=q;vcSendMsg()};
  window.vcClear=function(){
    msgs.innerHTML='<div class="vc-msg bot"><div class="vc-bbl">&#128075; Chat cleared. Ask me anything!</div></div>';
    chips.style.display='flex';
  };

  inp.addEventListener('keypress',function(e){if(e.key==='Enter')vcSendMsg()});
  inp.focus();
})();
</script>'''))